# MCQ Generation Fine-tuning with Unsloth GRPO
**Fine-tune Qwen2.5-3B-Instruct to generate Multiple Choice Questions from text context**

This notebook fine-tunes a model to:
- Take text passages as input
- Generate MCQ questions with 4 options and correct answer
- Output structured JSON format

Based on EleutherAI/RACE dataset format

## 1. Installation

In [1]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1"

# Install dependencies
!pip install --upgrade -qqq uv

try:
    import numpy, PIL
    get_numpy = f"numpy=={numpy.__version__}"
    get_pil = f"pillow=={PIL.__version__}"
except:
    get_numpy = "numpy"
    get_pil = "pillow"

try:
    import subprocess
    is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
except:
    is_t4 = False

get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")

!uv pip install -qqq --upgrade unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
!uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2
!uv pip install pydantic datasets

In [ ]:
# Mount Google Drive for persistent storage
from google.colab import drive
drive.mount('/content/drive')

# Create checkpoint directory in Google Drive
import os
checkpoint_dir = "/content/drive/MyDrive/mcq_grpo_checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

print(f"✓ Checkpoints will be saved to: {checkpoint_dir}")

## 2. Load Model

In [2]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

max_seq_length = 2048  # Increased for longer passages
lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.9,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

print("Model loaded successfully!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-16 17:29:16 [__init__.py:244] Automatically detected platform cuda.
ERROR 11-16 17:29:19 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 11-16 17:29:29 [vllm_utils.py:700] Unsloth: Patching vLLM v1 graph capture
INFO 11-16 17:29:29 [vllm_utils.py:730] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM 

`torch_dtype` is deprecated! Use `dtype` instead!


WARNING 11-16 17:29:50 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 11-16 17:29:50 [config.py:1472] Using max model len 2048
WARNING 11-16 17:29:51 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 11-16 17:29:53 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=2048.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection', 'model.layers.2.mlp', 'model.layers.3.mlp', 'model.layers.30.mlp'], 'llm_int8_threshold': 6.0}
INFO 11-16 17:29:53 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='unsloth/qwen2.5-3b-instruct-unsloth-b

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.11.3 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


Model loaded successfully!


## 3. Define Pydantic Output Schema

In [3]:
from pydantic import BaseModel, Field
from typing import List
import json

class MCQQuestion(BaseModel):
    """Single Multiple Choice Question"""
    question: str = Field(description="The question text")
    option_a: str = Field(description="Option A")
    option_b: str = Field(description="Option B")
    option_c: str = Field(description="Option C")
    option_d: str = Field(description="Option D")
    correct_answer: str = Field(description="Correct answer: A, B, C, or D")

class MCQList(BaseModel):
    """List of MCQ Questions"""
    questions: List[MCQQuestion] = Field(description="List of generated questions")

# Example JSON schema for reference
print("Expected JSON Output Format:")
print(json.dumps(MCQList.model_json_schema(), indent=2))

Expected JSON Output Format:
{
  "$defs": {
    "MCQQuestion": {
      "description": "Single Multiple Choice Question",
      "properties": {
        "question": {
          "description": "The question text",
          "title": "Question",
          "type": "string"
        },
        "option_a": {
          "description": "Option A",
          "title": "Option A",
          "type": "string"
        },
        "option_b": {
          "description": "Option B",
          "title": "Option B",
          "type": "string"
        },
        "option_c": {
          "description": "Option C",
          "title": "Option C",
          "type": "string"
        },
        "option_d": {
          "description": "Option D",
          "title": "Option D",
          "type": "string"
        },
        "correct_answer": {
          "description": "Correct answer: A, B, C, or D",
          "title": "Correct Answer",
          "type": "string"
        }
      },
      "required": [
        "question",

## 4. Prepare Dataset

In [8]:
from datasets import load_dataset

# Load one example to see the actual structure
dataset = load_dataset("EleutherAI/race", "high", split="test")
print("Dataset features:")
print(dataset.features)
print("\n" + "="*80)
print("First example keys:")
print(dataset[0].keys())
print("\n" + "="*80)
print("First example:")
for key, value in dataset[0].items():
    print(f"\n{key}:")
    print(f"  Type: {type(value)}")
    if isinstance(value, str):
        print(f"  Value (first 300 chars): {value[:300]}")
    else:
        print(f"  Value: {value}")


Dataset features:
{'article': Value('string'), 'problems': Value('string')}

First example keys:
dict_keys(['article', 'problems'])

First example:

article:
  Type: <class 'str'>
  Value (first 300 chars): The rain had continued for a week and the flood had created a big river which were running by Nancy Brown's farm. As she tried to gather her cows to a higher ground, she slipped and hit her head on a fallen tree trunk. The fall made her unconscious for a moment or two. When she came to, Lizzie, one 

problems:
  Type: <class 'str'>
  Value (first 300 chars): [{'question': 'What did Nancy try to do before she fell over?', 'answer': 'C', 'options': ['Measure the depth of the river', 'Look for a fallen tree trunk', 'Protect her cows from being drowned', 'Run away from the flooded farm']}, {'question': 'The following are true according to the passage except


In [9]:
from datasets import load_dataset, Dataset, concatenate_datasets
import re
import json
import ast  # Added to parse Python string representation

# System prompt for MCQ generation
SYSTEM_PROMPT = """You are an expert MCQ question generator. Given a text passage, generate multiple choice questions.

Output Format:
<questions>
[
  {
    "question": "Question text here?",
    "option_a": "First option",
    "option_b": "Second option",
    "option_c": "Third option",
    "option_d": "Fourth option",
    "correct_answer": "A"
  }
]
</questions>
"""

# Load RACE dataset
def prepare_race_dataset(split="test", num_samples=500, config="high"):
    """
    Convert RACE dataset to MCQ generation format

    Args:
        split: "test" (only available split for EleutherAI/race)
        num_samples: number of samples to use
        config: "high" (high school) or "middle" (middle school) or "both"
    """
    # Load dataset based on config
    if config == "both":
        dataset_high = load_dataset("EleutherAI/race", "high", split=split)
        dataset_middle = load_dataset("EleutherAI/race", "middle", split=split)
        dataset = concatenate_datasets([dataset_high, dataset_middle])
    else:
        dataset = load_dataset("EleutherAI/race", config, split=split)

    prepared_data = []

    for idx, example in enumerate(dataset):
        if idx >= num_samples:
            break

        article = example['article']
        # Use ast.literal_eval instead of json.loads because problems uses Python syntax (single quotes)
        problems = ast.literal_eval(example['problems'])

        # Build expected output
        mcq_list = []
        for problem in problems:
            question_text = problem['question']
            options = problem['options']
            answer = problem['answer']

            mcq_item = {
                "question": question_text,
                "option_a": options[0] if len(options) > 0 else "",
                "option_b": options[1] if len(options) > 1 else "",
                "option_c": options[2] if len(options) > 2 else "",
                "option_d": options[3] if len(options) > 3 else "",
                "correct_answer": answer
            }
            mcq_list.append(mcq_item)

        # Format as expected output
        expected_output = "<questions>\n" + json.dumps(mcq_list, indent=2) + "\n</questions>"

        prepared_data.append({
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': f"Generate MCQ questions from this text:\n\n{article}"}
            ],
            'expected_output': expected_output,
            'num_questions': len(mcq_list)
        })

    return Dataset.from_list(prepared_data)

# Load training data
print("Loading and preparing dataset...")
train_dataset = prepare_race_dataset(split="test", num_samples=500, config="high")

print(f"Prepared {len(train_dataset)} training examples")
print(f"\nExample entry:")
print(f"Prompt: {train_dataset[0]['prompt'][1]['content'][:200]}...")
print(f"\nExpected Output: {train_dataset[0]['expected_output'][:300]}...")


Loading and preparing dataset...
Prepared 500 training examples

Example entry:
Prompt: Generate MCQ questions from this text:

The rain had continued for a week and the flood had created a big river which were running by Nancy Brown's farm. As she tried to gather her cows to a higher gr...

Expected Output: <questions>
[
  {
    "question": "What did Nancy try to do before she fell over?",
    "option_a": "Measure the depth of the river",
    "option_b": "Look for a fallen tree trunk",
    "option_c": "Protect her cows from being drowned",
    "option_d": "Run away from the flooded farm",
    "correct_...


## 5. Define Reward Functions

In [10]:
import json
import re

def extract_json_content(text: str) -> str:
    """Extract JSON from <questions> tags"""
    match = re.search(r'<questions>\s*(.+?)\s*</questions>', text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return ""

def format_reward_func(completions, **kwargs) -> list[float]:
    """Reward for proper XML format with <questions> tags"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []

    for response in responses:
        reward = 0.0
        if '<questions>' in response:
            reward += 0.25
        if '</questions>' in response:
            reward += 0.25
        rewards.append(reward)

    return rewards

def json_validity_reward_func(completions, **kwargs) -> list[float]:
    """Reward for valid JSON structure"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []

    for response in responses:
        json_content = extract_json_content(response)
        try:
            parsed = json.loads(json_content)
            if isinstance(parsed, list):
                rewards.append(0.5)
            else:
                rewards.append(0.0)
        except:
            rewards.append(0.0)

    return rewards

def mcq_structure_reward_func(completions, **kwargs) -> list[float]:
    """Reward for proper MCQ structure"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []

    required_fields = ['question', 'option_a', 'option_b', 'option_c', 'option_d', 'correct_answer']

    for response in responses:
        json_content = extract_json_content(response)
        try:
            parsed = json.loads(json_content)
            if isinstance(parsed, list) and len(parsed) > 0:
                # Check if first question has all required fields
                first_q = parsed[0]
                if all(field in first_q for field in required_fields):
                    rewards.append(1.0)
                else:
                    rewards.append(0.0)
            else:
                rewards.append(0.0)
        except:
            rewards.append(0.0)

    return rewards

def answer_validity_reward_func(completions, **kwargs) -> list[float]:
    """Reward for correct answer format (A, B, C, or D)"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []

    for response in responses:
        json_content = extract_json_content(response)
        try:
            parsed = json.loads(json_content)
            if isinstance(parsed, list) and len(parsed) > 0:
                valid_answers = sum(1 for q in parsed if q.get('correct_answer', '') in ['A', 'B', 'C', 'D'])
                reward = valid_answers / len(parsed) * 0.5
                rewards.append(reward)
            else:
                rewards.append(0.0)
        except:
            rewards.append(0.0)

    return rewards

def question_count_reward_func(completions, num_questions, **kwargs) -> list[float]:
    """Reward for generating appropriate number of questions"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []

    for response, expected_count in zip(responses, num_questions):
        json_content = extract_json_content(response)
        try:
            parsed = json.loads(json_content)
            if isinstance(parsed, list):
                actual_count = len(parsed)
                # Reward based on how close to expected count
                if actual_count == expected_count:
                    rewards.append(0.5)
                elif abs(actual_count - expected_count) <= 1:
                    rewards.append(0.25)
                else:
                    rewards.append(0.0)
            else:
                rewards.append(0.0)
        except:
            rewards.append(0.0)

    return rewards

print("Reward functions defined!")

Reward functions defined!


## 6. Configure GRPO Training

In [ ]:
from trl import GRPOConfig, GRPOTrainer

# IMPORTANT: Point output_dir to Google Drive
training_args = GRPOConfig(
    use_vllm = True,
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 5,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    num_generations = 4,
    max_prompt_length = 1024,
    max_completion_length = 1024,
    max_steps = 300,
    save_steps = 50,  # Save every 50 steps (≈20 minutes)
    save_total_limit = 2,  # Keep only last 2 checkpoints to save space
    max_grad_norm = 0.1,
    report_to = "none",
    output_dir = "/content/drive/MyDrive/mcq_grpo_checkpoints",  # ← Google Drive!
)

print("✓ Training configuration ready!")


Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4
Training configuration ready!


## 7. Train the Model

**Training will take approximately 1-2 hours. Monitor the reward column - it should increase over time.**

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        format_reward_func,
        json_validity_reward_func,
        mcq_structure_reward_func,
        answer_validity_reward_func,
        question_count_reward_func,
    ],
    args = training_args,
    train_dataset = train_dataset,
)

print("Starting training...")

# Auto-resume from latest checkpoint if exists
import os
checkpoint_dir = "/content/drive/MyDrive/mcq_grpo_checkpoints"

checkpoints = []
if os.path.exists(checkpoint_dir):
    checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith("checkpoint-")]

if checkpoints:
    # Get latest checkpoint
    latest_checkpoint = sorted(checkpoints, key=lambda x: int(x.split("-")[1]))[-1]
    checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
    print(f"🔄 Resuming from checkpoint: {latest_checkpoint}")
    trainer.train(resume_from_checkpoint=checkpoint_path)
else:
    print("🆕 Starting fresh training")
    trainer.train()

print("✓ Training session complete!")


Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 3 | Total steps = 300
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 119,734,272 of 3,205,672,960 (3.74% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / format_reward_func / mean,rewards / format_reward_func / std,rewards / json_validity_reward_func / mean,rewards / json_validity_reward_func / std,rewards / mcq_structure_reward_func / mean,rewards / mcq_structure_reward_func / std,rewards / answer_validity_reward_func / mean,rewards / answer_validity_reward_func / std,rewards / question_count_reward_func / mean,rewards / question_count_reward_func / std
5,-0.000000,2.398750,0.314646,406.475000,289.400000,578.000000,0.000000,406.475000,289.400000,578.000000,0.000005,0.500000,0.000000,0.493750,0.025000,0.975000,0.068313,0.211250,0.251326,0.218750,0.192965
10,0.000000,2.278125,0.461678,432.000000,302.800000,574.800000,0.000000,432.000000,302.800000,574.800000,0.000013,0.500000,0.000000,0.468750,0.109157,0.925000,0.230623,0.187500,0.238737,0.196875,0.183333
15,0.000000,2.425000,0.394640,443.437500,285.000000,652.800000,0.000000,443.437500,285.000000,652.800000,0.000082,0.500000,0.000000,0.481250,0.059157,0.962500,0.118313,0.300000,0.248001,0.181250,0.173995
20,0.000000,2.525000,0.329383,435.200000,282.200000,739.800000,0.012500,427.719171,282.200000,696.000000,0.000679,0.496875,0.012500,0.481250,0.040311,0.937500,0.180623,0.362500,0.188183,0.246875,0.176655
25,0.000000,2.475000,0.468277,428.275000,336.800000,583.400000,0.000000,428.275000,336.800000,583.400000,0.003225,0.500000,0.000000,0.462500,0.118313,0.925000,0.236626,0.418750,0.184500,0.168750,0.155391
30,0.000000,2.556250,0.273330,410.962500,229.200000,647.200000,0.000000,410.962500,229.200000,647.200000,0.002858,0.493750,0.025000,0.481250,0.075000,0.962500,0.150000,0.462500,0.099468,0.156250,0.187661
35,0.000000,2.546875,0.428769,399.025000,254.800000,576.000000,0.000000,399.025000,254.800000,576.000000,0.004888,0.500000,0.000000,0.462500,0.118313,0.925000,0.236626,0.462500,0.118313,0.196875,0.191868
40,0.000000,2.709375,0.123922,382.475000,228.000000,555.800000,0.000000,382.475000,228.000000,555.800000,0.005955,0.500000,0.000000,0.500000,0.000000,1.000000,0.000000,0.500000,0.000000,0.209375,0.170399
45,0.000000,2.668750,0.284708,367.787500,242.600000,539.400000,0.000000,367.787500,242.600000,539.400000,0.007701,0.500000,0.000000,0.481250,0.075000,0.962500,0.150000,0.481250,0.075000,0.243750,0.171009
50,0.000000,2.756250,0.180439,312.062500,166.200000,462.400000,0.000000,312.062500,166.200000,462.400000,0.012055,0.500000,0.000000,0.493750,0.025000,0.987500,0.050000,0.493750,0.025000,0.281250,0.173132


## 8. Save LoRA Adapters

In [ ]:
# Save locally to Google Drive
lora_save_path = "/content/drive/MyDrive/mcq_grpo_lora_final"
model.save_lora(lora_save_path)
print(f"✓ LoRA adapters saved to Google Drive: {lora_save_path}")

# Optional: Upload to Hugging Face
from getpass import getpass

upload_to_hf = input("Upload to HuggingFace? (y/n): ").lower() == 'y'

if upload_to_hf:
    print("Enter your Hugging Face token (input will be hidden):")
    HF_TOKEN = getpass("HF Token: ")
    
    if HF_TOKEN and HF_TOKEN.startswith('hf_'):
        try:
            model.push_to_hub("mohamedashraff2222/qwen2.5-3b-mcq-lora", token=HF_TOKEN)
            tokenizer.push_to_hub("mohamedashraff2222/qwen2.5-3b-mcq-lora", token=HF_TOKEN)
            print("✓ LoRA adapters pushed to Hugging Face Hub!")
        except Exception as e:
            print(f"⚠️ Failed to push to HF: {e}")
    else:
        print("⚠️ Invalid token. Skipping HF upload.")


## 9. Test Inference - Base Model (Before Training)

In [ ]:
from vllm import SamplingParams

# Test with a sample passage
test_passage = """The rain had continued for a week and the flood had created a big river which were running by Nancy Brown's farm.
As she tried to gather her cows to a higher ground, she slipped and hit her head on a fallen tree trunk.
The fall made her unconscious for a moment or two. When she came to, Lizzie, one of her oldest and favorite cows, was licking her face.
At that time, the water level on the farm was still rising. Nancy gathered all her strength to get up and began walking slowly with Lizzie."""

test_prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": f"Generate MCQ questions from this text:\n\n{test_passage}"}
], tokenize=False, add_generation_prompt=True)

sampling_params = SamplingParams(
    temperature = 0.7,
    top_p = 0.9,
    max_tokens = 1024,
)

print("=" * 80)
print("BASE MODEL OUTPUT (Without LoRA):")
print("=" * 80)

output_base = model.fast_generate(
    [test_prompt],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

print(output_base)
print("\n" + "=" * 80)

## 10. Test Inference - Fine-tuned Model with LoRA

In [ ]:
# Load the LoRA adapters
lora_adapter = model.load_lora("mcq_grpo_lora")

print("=" * 80)
print("FINE-TUNED MODEL OUTPUT (With LoRA):")
print("=" * 80)

output_finetuned = model.fast_generate(
    test_prompt,
    sampling_params = sampling_params,
    lora_request = lora_adapter,
)[0].outputs[0].text

print(output_finetuned)
print("\n" + "=" * 80)

## 11. Parse and Validate Output

In [ ]:
def parse_mcq_output(output_text: str) -> dict:
    """Parse and validate MCQ output"""
    json_content = extract_json_content(output_text)

    try:
        parsed = json.loads(json_content)

        # Validate with Pydantic
        mcq_list = MCQList(questions=[MCQQuestion(**q) for q in parsed])

        return {
            "success": True,
            "data": mcq_list.model_dump(),
            "num_questions": len(mcq_list.questions)
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "raw_output": output_text[:500]
        }

# Parse fine-tuned output
result = parse_mcq_output(output_finetuned)

if result["success"]:
    print("✅ Successfully generated and validated MCQ questions!")
    print(f"\nGenerated {result['num_questions']} questions:")
    print(json.dumps(result["data"], indent=2))
else:
    print("❌ Failed to parse output:")
    print(f"Error: {result['error']}")
    print(f"Raw output: {result['raw_output']}")

## 12. Test with Large Custom Text

In [ ]:
# Example: Test with a longer passage (e.g., from a book)
large_text = """Machine learning is a subset of artificial intelligence that provides systems the ability to automatically learn
and improve from experience without being explicitly programmed. Machine learning focuses on the development of computer programs
that can access data and use it to learn for themselves. The process of learning begins with observations or data, such as examples,
direct experience, or instruction, in order to look for patterns in data and make better decisions in the future based on the examples
that we provide. The primary aim is to allow the computers to learn automatically without human intervention or assistance and adjust
actions accordingly.

There are several types of machine learning algorithms. Supervised learning algorithms can apply what has been learned in the past to new data
using labeled examples to predict future events. Starting from the analysis of a known training dataset, the learning algorithm produces an
inferred function to make predictions about the output values. The system is able to provide targets for any new input after sufficient training.

Unsupervised learning is used when the information used to train is neither classified nor labeled. Unsupervised learning studies how systems
can infer a function to describe a hidden structure from unlabeled data. The system doesn't figure out the right output, but it explores the
data and can draw inferences from datasets to describe hidden structures from unlabeled data."""

large_prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": f"Generate MCQ questions from this text:\n\n{large_text}"}
], tokenize=False, add_generation_prompt=True)

print("=" * 80)
print("GENERATING MCQs FROM LARGE TEXT:")
print("=" * 80)

output_large = model.fast_generate(
    large_prompt,
    sampling_params = SamplingParams(
        temperature = 0.7,
        top_p = 0.9,
        max_tokens = 1536,  # Increased for more questions
    ),
    lora_request = lora_adapter,
)[0].outputs[0].text

print(output_large)
print("\n" + "=" * 80)

# Parse and validate
result_large = parse_mcq_output(output_large)
if result_large["success"]:
    print(f"\n✅ Successfully generated {result_large['num_questions']} questions from large text!")

    # Save to JSON file
    with open('generated_mcqs.json', 'w') as f:
        json.dump(result_large["data"], f, indent=2)
    print("📁 Saved to 'generated_mcqs.json'")
else:
    print(f"\n❌ Parsing failed: {result_large['error']}")

## 13. Load LoRA from Hugging Face Hub (Optional)

**Use this section to load previously uploaded LoRA adapters**

In [ ]:
# Uncomment to load from Hugging Face Hub
"""
from huggingface_hub import hf_hub_download
import os

# Download LoRA adapters
HF_MODEL_ID = "your-username/qwen2.5-3b-mcq-lora"
HF_TOKEN = "your_hf_token_here"  # Optional if public repo

# Create directory for downloaded adapters
os.makedirs("downloaded_lora", exist_ok=True)

# Download adapter files
adapter_files = [
    "adapter_config.json",
    "adapter_model.safetensors"
]

for file in adapter_files:
    hf_hub_download(
        repo_id=HF_MODEL_ID,
        filename=file,
        local_dir="downloaded_lora",
        token=HF_TOKEN
    )

print("LoRA adapters downloaded from Hugging Face!")

# Load the downloaded adapters
lora_adapter_hf = model.load_lora("downloaded_lora")
print("LoRA adapters loaded and ready for inference!")
"""
pass

## 14. Inference Helper Function

In [ ]:
def generate_mcqs_from_text(text: str, num_questions: int = None) -> dict:
    """
    Generate MCQ questions from input text

    Args:
        text: Input text passage
        num_questions: Optional number of questions to generate

    Returns:
        Dictionary with success status and generated questions
    """
    user_content = f"Generate MCQ questions from this text:\n\n{text}"
    if num_questions:
        user_content += f"\n\nGenerate exactly {num_questions} questions."

    prompt = tokenizer.apply_chat_template([
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_content}
    ], tokenize=False, add_generation_prompt=True)

    output = model.fast_generate(
        prompt,
        sampling_params = SamplingParams(
            temperature = 0.7,
            top_p = 0.9,
            max_tokens = 1536,
        ),
        lora_request = lora_adapter,
    )[0].outputs[0].text

    return parse_mcq_output(output)

# Test the helper function
test_text = """Python is a high-level, interpreted programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms,
including procedural, object-oriented, and functional programming."""

result = generate_mcqs_from_text(test_text, num_questions=3)

if result["success"]:
    print("✅ Generated MCQs:")
    for i, q in enumerate(result["data"]["questions"], 1):
        print(f"\n{i}. {q['question']}")
        print(f"   A) {q['option_a']}")
        print(f"   B) {q['option_b']}")
        print(f"   C) {q['option_c']}")
        print(f"   D) {q['option_d']}")
        print(f"   Correct Answer: {q['correct_answer']}")
else:
    print(f"❌ Error: {result['error']}")

## 15. Batch Processing - Generate MCQs from Multiple Texts

In [ ]:
def batch_generate_mcqs(texts: List[str], output_file: str = "batch_mcqs.json") -> dict:
    """
    Generate MCQs from multiple text passages

    Args:
        texts: List of text passages
        output_file: Output JSON filename

    Returns:
        Dictionary with results for each text
    """
    results = []

    for idx, text in enumerate(texts):
        print(f"Processing text {idx+1}/{len(texts)}...")
        result = generate_mcqs_from_text(text)

        results.append({
            "text_id": idx,
            "text_preview": text[:100] + "...",
            "success": result["success"],
            "questions": result.get("data", {}).get("questions", []),
            "num_questions": result.get("num_questions", 0)
        })

    # Save to file
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)

    print(f"\n✅ Batch processing complete! Saved to '{output_file}'")
    return results

# Example batch processing
sample_texts = [
    """Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed
    to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits
    associated with a human mind such as learning and problem-solving.""",

    """Deep learning is part of a broader family of machine learning methods based on artificial neural networks with
    representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep learning architectures
    such as deep neural networks have been applied to fields including computer vision and natural language processing."""
]

batch_results = batch_generate_mcqs(sample_texts)

# Display summary
print(f"\nBatch Summary:")
for result in batch_results:
    status = "✅" if result["success"] else "❌"
    print(f"{status} Text {result['text_id']}: {result['num_questions']} questions generated")

## Summary

This notebook demonstrates:
1. ✅ Fine-tuning Qwen2.5-3B-Instruct with GRPO for MCQ generation
2. ✅ Using Pydantic models for structured output validation
3. ✅ Training on RACE dataset format
4. ✅ Saving LoRA adapters (not merged with base model)
5. ✅ Loading adapters from Hugging Face Hub
6. ✅ Generating MCQs from custom text passages
7. ✅ Batch processing multiple texts
8. ✅ Exporting results to JSON files

**Next Steps:**
- Upload your LoRA adapters to Hugging Face Hub
- Test with your own book chapters
- Adjust reward functions for better quality
- Extend training for improved performance